In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [3]:
def write_json_to_txt(json_data, file_name):
    filtered_data = {k: v for k, v in json_data.items() if v.get("reflect") == "yes"}
    with open(file_name, 'w') as file:
        for test_case, details in filtered_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [4]:
SYSTEM_PROMPT_8="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  ],
}
If there are more than one test case for this scenario, continue writing other test case in this form.

Rules for generating test steps:
- Describe the test step clearly to make sure each test case is independent, tester do not need to read other information (example: other test case, use case specification) to know how to do that step.
- Avoid references to other test cases or instructions like "do as mentioned."
- If the test case need to be repeated to test with different order, data or case, seperate them to be distinct test cases.
- If the scenario is about testing the displation and there is no flow directly cover that scenario, use only the basic (or main) flow to test it.
- If there are use cases mentioned in extended or included use case, create test case combine use cases, Try to find the connection point of use cases for combination. 
- For test scenarios mentioning navigation in the name, only produce test cases related to the specified navigation method. 
(Example: "Scenario: User navigates to a page by navbar", only produce a test case of user navigates to that page by navbar even though the use case description has many way to navigate to that page)
- For test scenarios not mentioning navigation in the name, do not include any navigation test cases.
- Generate test cases that directly match the scenario name. Choose only one flow to cover the scenario.
- Ensure all actions and objectives match the scenario name.
"""

In [5]:
SELF_REF="""
Given a test scenario and test cases to test that given test scenario.
Mark if test case can test the given test scenario or not through test steps, expected output, objective (although if it test other use case path or flow, if it is not used to test the given test scenario.)
And give explanation why you think the resulted test case reflect the given test scenario or not. 
Your response should keep the format of the inserted test cases.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "reflect": "yes/no",
    "explanation": "explain why you think this test case reflect the given test scenario or not",
  ],
}"""

In [6]:
scenarios = ["Successful Lesson and Topic Selection with Flashcard Review Completion", 
             "Navigation to Previous Flashcard", "Learner Decides to Stay After Accidentally Pressing 'Back' Button During Flashcard Study",
             "Learner chooses to 'continue studying' after completing a topic", 
             "Learner continues an incomplete topic from the Home Page", "Attempt to Navigate to Previous Flashcard on First Flashcard",
             "Learner exits flashcard study via back button"]

In [7]:
prompt_uc = """
use case id: 
use case name: flashcard
objective: This use case allows learners to study vocabulary words through interactive flashcards categorized by topics and lessons. Learners can navigate through different topics within a lesson and flip through flashcards to learn new words and their meanings.
actor: learner 
Preconditions:
Learner must be logged into the application.

basic flow:
Step 1: Learner clicks the "Learn New Word" tab on the navigation bar.
Step 2:The system displays a list of available lessons.
Step 3:Learner selects a lesson from the list.
Step 4:The system displays the details of the selected lesson, including the topics associated with it.
Step 5:Learner chooses a topic to study.
Step 6:The system redirects the learner to the corresponding Flashcard page for the selected topic.
Step 7:The system displays the first flashcard of the vocabulary words for the selected topic. The front of the flashcard shows the vocabulary name, pronunciation, and Vietnamese meaning.
Step 8:Learner click on the flashcard to flip it and view the back, which contains illustrations and examples of real-life usage.
Step 9:Learner click the "Next" button to go to the flashcard of the next vocabulary word.
Step 10:If there is a flashcard of the previous word, learner click the "Previous" button to go back to the previous flashcard.
Step 11:Steps 8-11 are repeated for all flashcards in the topic.
Step 12:Once all the flashcards have been studied, the 'Next' button on the last flashcard will redirect system to the 'completion notification' page with a congratulatory message and option buttons to 'continue studying' and 'return to the lesson page'.
Step 13: learner clicks "return to the lesson page" 
Step 14: the system redirect to the lesson page.

Alternative Flows:
Alternative flow 1: Learner presses the 'back' button during flashcard study and presses 'stay'.
At step 7 of the basic flow: Learner presses the 'back button.
Step 8: Exit confirmation dialog box appears, asking "Are you sure you want to exit?"
Step 9: Learner clicks 'stay'.
Go back to step 7 in the basic flow and continue with the steps from step 7.

Alternative flow 2:Learner presses the 'continue studying'
At step 13 of the basic flow: learner clicks 'continue studying'
Step 14: Topic is marked as completed. Vocabulary words are marked as studied.Learner can review completed topics and studied vocabulary later.
Step 15: The system redirects the learner to the corresponding Flashcard page for the continuous topic.

Alternative flow 3: Continuing from Home Page with incomplete topic displayed on the home page
At step 1 of the basic flow: Learner clicks one incomplete topic.
Go back to step 6 in the basic flow and continue with the steps from step 6.

Alternative flow 4: learner tries to click 'Previous' button when on the first flashcard, the button is disabled.
At step 9 of the basic flow: Learner clicks 'Previous' button.
Step 10: the system display message 'there is no previous vocabulary to display'.
Go back to step 9 in the basic flow and continue with the steps from step 9.

Exception flow:
Exception flow 1: Learner presses the 'back' button during flashcard study and presses 'exit'.
At step 7 of the basic flow: Learner presses the 'back' button.
Step 8: Exit confirmation dialog box appears, asking "Are you sure you want to exit?"
Step 9: Learner clicks 'exit'.
Step 10: System returns learner to the topic page and the topic is marked as incomplete topic and be displayed on the home page.
Postcondition: 
Topic is marked as completed. 
Vocabulary words are marked as studied.
Learner can review completed topics and studied vocabulary later.
"""

In [8]:
for scenario in scenarios:
    i = 1
    while(i<=1):
        try: 
            promptTestCase = [
            { "role": "system", "content": SYSTEM_PROMPT_8},
            { "role": "user", "content": scenario + "\n" + prompt_uc}
            ]
            gpt_response = ask(promptTestCase, client, model)
            json_data = json.loads(gpt_response)
            write_json_to_txt_raw(json_data, scenario + "-Raw" + str(i) + ".txt")

            promptSelfRef = [
                { "role": "system", "content": SELF_REF},
                { "role": "user", "content": scenario + "\n" + gpt_response}
            ]
            final_response = ask(promptSelfRef, client, model)
            json_fin = json.loads(final_response)
            write_json_to_txt(json_fin, scenario + str(i) + ".txt")
            i+=1
        except Exception as e:
            print(f"An error occurred with scenario '{scenario}' iteration {i}: {e}")